In [15]:
from dotenv import load_dotenv
import os
import asyncio
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient

In [16]:
load_dotenv()

True

In [22]:
TENANT_ID = os.getenv("TENANT_ID")

In [23]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [25]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID)
scopes = ['User.Read', 'Mail.Read']
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [28]:
async def get_user():
    user = await client.users.by_user_id('adelev@fabster.onmicrosoft.com').get()
    if user:
        print(user.display_name)

In [29]:
await (get_user())

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AZGJSP4XT to authenticate.


DeviceCodeCredential.get_token failed: Authentication failed: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'. Trace ID: 3540d8f6-cc8f-44ef-adcc-6d25f874c500 Correlation ID: f2ccb349-ac20-4de5-973b-77e0f8c6454a Timestamp: 2023-11-14 03:33:48Z
Content: {"error":"invalid_client","error_description":"AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'. Trace ID: 3540d8f6-cc8f-44ef-adcc-6d25f874c500 Correlation ID: f2ccb349-ac20-4de5-973b-77e0f8c6454a Timestamp: 2023-11-14 03:33:48Z","error_codes":[7000218],"timestamp":"2023-11-14 03:33:48Z","trace_id":"3540d8f6-cc8f-44ef-adcc-6d25f874c500","correlation_id":"f2ccb349-ac20-4de5-973b-77e0f8c6454a","error_uri":"https://login.microsoftonline.com/error?code=7000218"}


ClientAuthenticationError: Authentication failed: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'. Trace ID: 3540d8f6-cc8f-44ef-adcc-6d25f874c500 Correlation ID: f2ccb349-ac20-4de5-973b-77e0f8c6454a Timestamp: 2023-11-14 03:33:48Z
Content: {"error":"invalid_client","error_description":"AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'. Trace ID: 3540d8f6-cc8f-44ef-adcc-6d25f874c500 Correlation ID: f2ccb349-ac20-4de5-973b-77e0f8c6454a Timestamp: 2023-11-14 03:33:48Z","error_codes":[7000218],"timestamp":"2023-11-14 03:33:48Z","trace_id":"3540d8f6-cc8f-44ef-adcc-6d25f874c500","correlation_id":"f2ccb349-ac20-4de5-973b-77e0f8c6454a","error_uri":"https://login.microsoftonline.com/error?code=7000218"}